### Census Tract Issue - USDA Food Access Research Atlas (Tennessee data only)

In [443]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


Importing both datasets

In [444]:
tn_food_access_df = pd.read_csv('../data/tn_food_access.csv')

In [445]:
tn_food_access_df = tn_food_access_df.drop('State', axis=1)

In [446]:
# I need to see if this is doing something constructive with the data, or anything at all, before running:
# tn_food_access_df = tn_food_access_df.replace(r'^\s*$', np.nan, regex=True)

In [447]:
# tn_food_access_df_fillna.to_csv('tn_food_access_nan.csv')

Getting a general sense of the way census tracts operate in the two datasets ...

In [448]:
tn_food_access_df.head()

,CT_Old,CT_Clean,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,47001020100,201.00,Anderson County,1,3111,1546,0,6,0.19,0,...,384,1969,853,130,2,9,148,145,206,438
1,47001020201,202.01,Anderson County,1,3670,1535,0,103,2.81,0,...,658,3158,144,256,0,6,106,84,49,31
2,47001020202,202.02,Anderson County,1,4507,2025,0,230,5.10,0,...,1364,3988,194,128,0,18,179,214,174,208
3,47001020300,203.00,Anderson County,1,3822,1656,0,0,0.00,0,...,640,3290,278,20,3,18,213,179,53,109
4,47001020400,204.00,Anderson County,1,4508,2013,0,106,2.35,1,...,761,3855,264,34,3,20,332,277,86,406


In [449]:
tn_food_access_df['CT_Clean'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1489 entries, 0 to 1488
Series name: CT_Clean
Non-Null Count  Dtype  
--------------  -----  
1489 non-null   float64
dtypes: float64(1)
memory usage: 11.8 KB


993 unique entries for census tract, yet 1,489 total? Need to look into this as a top priority.

In [450]:
tn_food_access_df['CT_Clean'].nunique()

993

In [451]:
# https://saturncloud.io/blog/how-to-check-for-duplicate-values-in-pandas-dataframe-column/#:~:text=To%20check%20for%20duplicate%20values%20in%20a%20Pandas%20DataFrame%20column,is%20a%20duplicate%20or%20not.
# duplicate_values = df['Product'].duplicated()
# print(duplicate_values)

duplicate_CTs = tn_food_access_df['CT_Clean'].duplicated()
duplicate_CTs_to_csv = pd.DataFrame(duplicate_CTs)
# duplicate_CTs_to_csv.to_csv('duplicate_CTs.csv')


ISSUE: converting the census tract numbers to match the census reporter data did not work and created duplicate census tract IDs in about 1/3 of the data. I will probably need to go back and change some things but need some help troubleshooting and figuring out where to start/what's needed.

In [452]:
tn_food_access_df['CT_Old'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1489 entries, 0 to 1488
Series name: CT_Old
Non-Null Count  Dtype
--------------  -----
1489 non-null   int64
dtypes: int64(1)
memory usage: 11.8 KB


In [453]:
tn_food_access_df['CT_Old'].nunique()

1489

#### Origin of issue discovery:

Pulling minimum population, I'm seeing two lines that appear to have the same census tract number. 

In [454]:
tn_food_access_df[tn_food_access_df['Pop2010'].iloc[:]==tn_food_access_df['Pop2010'].iloc[:].min()]

,CT_Old,CT_Clean,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
529,47065980100,801.0,Hamilton County,1,1,1,0,0,0.0,0,...,0,1,0,0,0,0,0,0,0,0
1043,47155980100,801.0,Sevier County,0,1,1,0,0,0.0,0,...,0,1,0,0,0,0,0,0,0,0


After speaking with instructors:
* pull digits only from census_df
* pull out county, create column for that
* make a list out of all column headers
* expand/true on comma
* take CT_Old column from tn_food_access_df and compare
* be sure to add "from income" to census_df and "from food" to tn_food_access_df
* union and catch for duplicates
* see the stuff that only exists in one form or another
* maybe also research census tract numbers (do they come in other forms? why wouldn't they be the same? is one more granular?)

In [455]:
census_df = pd.read_excel('../data/census_tn_income_b19001.xlsx')

In [456]:
census_df.head()

,Unnamed: 0,Tennessee,Unnamed: 2,"Census Tract 201, Anderson, TN",Unnamed: 4,"Census Tract 202.01, Anderson, TN",Unnamed: 6,"Census Tract 202.02, Anderson, TN",Unnamed: 8,"Census Tract 203, Anderson, TN",...,"Census Tract 309.05, Wilson, TN",Unnamed: 3396,"Census Tract 309.06, Wilson, TN",Unnamed: 3398,"Census Tract 309.07, Wilson, TN",Unnamed: 3400,"Census Tract 309.08, Wilson, TN",Unnamed: 3402,"Census Tract 310, Wilson, TN",Unnamed: 3404
0,NaN,Value,Error,Value,Error,Value,Error,Value,Error,Value,...,Value,Error,Value,Error,Value,Error,Value,Error,Value,Error
1,Total:,2713635,7405,1531,225,2029,160,1951,235,1567,...,1925,288,1963,244,2513,276,3124,432,2365,373
2,"Less than $10,000",144639,2807,156,90,93,73,235,127,25,...,38,64,0,19,109,89,133,92,159,106
3,"$10,000 to $14,999",122212,3595,74,52,28,35,159,126,95,...,0,19,21,35,60,58,61,102,174,158
4,"$15,000 to $19,999",111419,2673,41,42,26,30,7,13,56,...,0,19,17,30,59,68,46,52,124,75


In [457]:
census_df_ct_troubleshoot = census_df.rename(columns={'Unnamed: 0':'income_bracket'})

In [458]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot[census_df_ct_troubleshoot.columns.drop(list(census_df_ct_troubleshoot.filter(regex='Unnamed')))]


In [459]:
census_df_ct_troubleshoot.head()

,income_bracket,Tennessee,"Census Tract 201, Anderson, TN","Census Tract 202.01, Anderson, TN","Census Tract 202.02, Anderson, TN","Census Tract 203, Anderson, TN","Census Tract 204, Anderson, TN","Census Tract 205, Anderson, TN","Census Tract 206, Anderson, TN","Census Tract 207, Anderson, TN",...,"Census Tract 305, Wilson, TN","Census Tract 306, Wilson, TN","Census Tract 307, Wilson, TN","Census Tract 308, Wilson, TN","Census Tract 309.04, Wilson, TN","Census Tract 309.05, Wilson, TN","Census Tract 309.06, Wilson, TN","Census Tract 309.07, Wilson, TN","Census Tract 309.08, Wilson, TN","Census Tract 310, Wilson, TN"
0,NaN,Value,Value,Value,Value,Value,Value,Value,Value,Value,...,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value
1,Total:,2713635,1531,2029,1951,1567,1812,1615,926,586,...,3036,1678,1427,2533,1503,1925,1963,2513,3124,2365
2,"Less than $10,000",144639,156,93,235,25,104,133,13,22,...,94,82,235,80,91,38,0,109,133,159
3,"$10,000 to $14,999",122212,74,28,159,95,59,139,24,82,...,131,71,27,282,26,0,21,60,61,174
4,"$15,000 to $19,999",111419,41,26,7,56,88,94,3,34,...,223,53,112,93,30,0,17,59,46,124


In [460]:
# this removes the first row of data
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [461]:
census_df_ct_troubleshoot.head()

,income_bracket,Tennessee,"Census Tract 201, Anderson, TN","Census Tract 202.01, Anderson, TN","Census Tract 202.02, Anderson, TN","Census Tract 203, Anderson, TN","Census Tract 204, Anderson, TN","Census Tract 205, Anderson, TN","Census Tract 206, Anderson, TN","Census Tract 207, Anderson, TN",...,"Census Tract 305, Wilson, TN","Census Tract 306, Wilson, TN","Census Tract 307, Wilson, TN","Census Tract 308, Wilson, TN","Census Tract 309.04, Wilson, TN","Census Tract 309.05, Wilson, TN","Census Tract 309.06, Wilson, TN","Census Tract 309.07, Wilson, TN","Census Tract 309.08, Wilson, TN","Census Tract 310, Wilson, TN"
1,Total:,2713635,1531,2029,1951,1567,1812,1615,926,586,...,3036,1678,1427,2533,1503,1925,1963,2513,3124,2365
2,"Less than $10,000",144639,156,93,235,25,104,133,13,22,...,94,82,235,80,91,38,0,109,133,159
3,"$10,000 to $14,999",122212,74,28,159,95,59,139,24,82,...,131,71,27,282,26,0,21,60,61,174
4,"$15,000 to $19,999",111419,41,26,7,56,88,94,3,34,...,223,53,112,93,30,0,17,59,46,124
5,"$20,000 to $24,999",115858,50,74,13,30,40,44,44,58,...,184,70,160,72,0,36,28,30,0,39


In [462]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.transpose().reset_index()

In [463]:
census_df_ct_troubleshoot.columns = census_df_ct_troubleshoot.iloc[0]

In [464]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [465]:
census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', regex=True, inplace = True)

In [466]:
# census_df_ct_troubleshoot_test = census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', inplace=True)

I've successfully taken "Census Tract" out of the income_bracket column. Now I need to create a new column and use the .split() function to separate the income bracket column into two separate columns: one for the census tract number, and one for the county name. Then I probably want another column (instead of altering the original ct number) which removes the decimal points from the census tracts. 

In [467]:
census_df_ct_troubleshoot.head()

,income_bracket,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980
2,"201, Anderson, TN",1531,156,74,41,50,106,31,262,154,52,168,138,101,125,16,46,11
3,"202.01, Anderson, TN",2029,93,28,26,74,24,0,54,89,86,151,219,124,242,195,256,368
4,"202.02, Anderson, TN",1951,235,159,7,13,101,48,40,33,140,74,257,164,134,129,295,122
5,"203, Anderson, TN",1567,25,95,56,30,84,65,74,46,53,173,170,280,174,106,93,43


In [468]:
census_df_ct_troubleshoot.columns.values.tolist()

['income_bracket',
 'Total:',
 'Less than $10,000',
 '$10,000 to $14,999',
 '$15,000 to $19,999',
 '$20,000 to $24,999',
 '$25,000 to $29,999',
 '$30,000 to $34,999',
 '$35,000 to $39,999',
 '$40,000 to $44,999',
 '$45,000 to $49,999',
 '$50,000 to $59,999',
 '$60,000 to $74,999',
 '$75,000 to $99,999',
 '$100,000 to $124,999',
 '$125,000 to $149,999',
 '$150,000 to $199,999',
 '$200,000 or more']

In [469]:
# df[['First Name', 'Last Name']] = df['Name'].str.split(' ', expand=True)
# https://saturncloud.io/blog/how-to-split-one-column-into-multiple-columns-in-pandas-dataframe/

census_df_ct_troubleshoot[['income_bracket','county_cr']] = census_df_ct_troubleshoot['income_bracket'].str.split(',',n=1,expand=True)
census_df_ct_troubleshoot.head()



,income_bracket,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",county_cr
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980,None
2,201,1531,156,74,41,50,106,31,262,154,52,168,138,101,125,16,46,11,"Anderson, TN"
3,202.01,2029,93,28,26,74,24,0,54,89,86,151,219,124,242,195,256,368,"Anderson, TN"
4,202.02,1951,235,159,7,13,101,48,40,33,140,74,257,164,134,129,295,122,"Anderson, TN"
5,203,1567,25,95,56,30,84,65,74,46,53,173,170,280,174,106,93,43,"Anderson, TN"


In [470]:
census_df_ct_troubleshoot['county_cr'].replace(', TN','', regex=True, inplace = True)

In [471]:
census_df_ct_troubleshoot.head()

,income_bracket,Total:,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",county_cr
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980,None
2,201,1531,156,74,41,50,106,31,262,154,52,168,138,101,125,16,46,11,Anderson
3,202.01,2029,93,28,26,74,24,0,54,89,86,151,219,124,242,195,256,368,Anderson
4,202.02,1951,235,159,7,13,101,48,40,33,140,74,257,164,134,129,295,122,Anderson
5,203,1567,25,95,56,30,84,65,74,46,53,173,170,280,174,106,93,43,Anderson


In [472]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

In [473]:
census_df_ct_troubleshoot.head()

,census_tract,total,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",county_cr
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980,None
2,201,1531,156,74,41,50,106,31,262,154,52,168,138,101,125,16,46,11,Anderson
3,202.01,2029,93,28,26,74,24,0,54,89,86,151,219,124,242,195,256,368,Anderson
4,202.02,1951,235,159,7,13,101,48,40,33,140,74,257,164,134,129,295,122,Anderson
5,203,1567,25,95,56,30,84,65,74,46,53,173,170,280,174,106,93,43,Anderson


In [474]:
census_df_ct_troubleshoot['ct_digits_cr'] = census_df_ct_troubleshoot['census_tract'].str.replace('.','')

In [475]:
census_df_ct_troubleshoot.head(200)

,census_tract,total,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",county_cr,ct_digits_cr
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,114744,211329,267360,357378,257232,169567,182209,197980,None,Tennessee
2,201,1531,156,74,41,50,106,31,262,154,52,168,138,101,125,16,46,11,Anderson,201
3,202.01,2029,93,28,26,74,24,0,54,89,86,151,219,124,242,195,256,368,Anderson,20201
4,202.02,1951,235,159,7,13,101,48,40,33,140,74,257,164,134,129,295,122,Anderson,20202
5,203,1567,25,95,56,30,84,65,74,46,53,173,170,280,174,106,93,43,Anderson,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,9614,1446,27,69,181,128,59,89,64,48,20,80,101,154,186,124,76,40,Crockett,9614
197,9701.01,2480,166,122,50,35,67,117,80,137,13,270,253,491,489,65,30,95,Cumberland,970101
198,9701.03,1536,41,11,27,146,29,108,119,45,84,118,118,425,139,104,0,22,Cumberland,970103
199,9701.04,1492,37,19,21,76,59,181,12,119,106,144,123,325,123,39,93,15,Cumberland,970104


In [476]:
census_df_ct_troubleshoot['from'] = 'census'

In [477]:
census_df_ct_troubleshoot.head()

,census_tract,total,"Less than $10,000","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more",county_cr,ct_digits_cr,from
1,Tennessee,2713635,144639,122212,111419,115858,114639,115755,118563,112751,...,211329,267360,357378,257232,169567,182209,197980,None,Tennessee,census
2,201,1531,156,74,41,50,106,31,262,154,...,168,138,101,125,16,46,11,Anderson,201,census
3,202.01,2029,93,28,26,74,24,0,54,89,...,151,219,124,242,195,256,368,Anderson,20201,census
4,202.02,1951,235,159,7,13,101,48,40,33,...,74,257,164,134,129,295,122,Anderson,20202,census
5,203,1567,25,95,56,30,84,65,74,46,...,173,170,280,174,106,93,43,Anderson,203,census


In [478]:
tn_food_access_df.head()

,CT_Old,CT_Clean,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,47001020100,201.00,Anderson County,1,3111,1546,0,6,0.19,0,...,384,1969,853,130,2,9,148,145,206,438
1,47001020201,202.01,Anderson County,1,3670,1535,0,103,2.81,0,...,658,3158,144,256,0,6,106,84,49,31
2,47001020202,202.02,Anderson County,1,4507,2025,0,230,5.10,0,...,1364,3988,194,128,0,18,179,214,174,208
3,47001020300,203.00,Anderson County,1,3822,1656,0,0,0.00,0,...,640,3290,278,20,3,18,213,179,53,109
4,47001020400,204.00,Anderson County,1,4508,2013,0,106,2.35,1,...,761,3855,264,34,3,20,332,277,86,406


In [479]:
# census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

tn_food_access_df = tn_food_access_df.rename(columns={'CT_Old':'ct_long_usda','CT_Clean':'ct_error_usda','County':'county_usda'})

In [480]:
tn_food_access_df.head()

,ct_long_usda,ct_error_usda,county_usda,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,47001020100,201.00,Anderson County,1,3111,1546,0,6,0.19,0,...,384,1969,853,130,2,9,148,145,206,438
1,47001020201,202.01,Anderson County,1,3670,1535,0,103,2.81,0,...,658,3158,144,256,0,6,106,84,49,31
2,47001020202,202.02,Anderson County,1,4507,2025,0,230,5.10,0,...,1364,3988,194,128,0,18,179,214,174,208
3,47001020300,203.00,Anderson County,1,3822,1656,0,0,0.00,0,...,640,3290,278,20,3,18,213,179,53,109
4,47001020400,204.00,Anderson County,1,4508,2013,0,106,2.35,1,...,761,3855,264,34,3,20,332,277,86,406


In [481]:
# census_df_ct_troubleshoot['from'] = 'census'
tn_food_access_df['from'] = 'usda'

In [482]:
tn_food_access_df['county_usda'].replace(' County','', regex=True, inplace = True)

In [483]:
usda_data_to_union = tn_food_access_df[['ct_long_usda','county_usda','from']].copy()

In [484]:
tn_food_access_df.head()

,ct_long_usda,ct_error_usda,county_usda,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,from
0,47001020100,201.00,Anderson,1,3111,1546,0,6,0.19,0,...,1969,853,130,2,9,148,145,206,438,usda
1,47001020201,202.01,Anderson,1,3670,1535,0,103,2.81,0,...,3158,144,256,0,6,106,84,49,31,usda
2,47001020202,202.02,Anderson,1,4507,2025,0,230,5.10,0,...,3988,194,128,0,18,179,214,174,208,usda
3,47001020300,203.00,Anderson,1,3822,1656,0,0,0.00,0,...,3290,278,20,3,18,213,179,53,109,usda
4,47001020400,204.00,Anderson,1,4508,2013,0,106,2.35,1,...,3855,264,34,3,20,332,277,86,406,usda


In [491]:
usda_data_to_union = usda_data_to_union.rename(columns={'ct_long_usda':'ct','county_usda':'county'})

In [492]:
usda_data_to_union.head()

,ct,county,from
0,47001020100,Anderson,usda
1,47001020201,Anderson,usda
2,47001020202,Anderson,usda
3,47001020300,Anderson,usda
4,47001020400,Anderson,usda


In [486]:
cr_data_to_union = census_df_ct_troubleshoot[['ct_digits_cr','county_cr','from']].copy()

In [487]:
# this removes the first row of data
cr_data_to_union = cr_data_to_union.iloc[1:,:]

,ct_digits_cr,county_cr,from
2,201,Anderson,census
3,20201,Anderson,census
4,20202,Anderson,census
5,203,Anderson,census
6,204,Anderson,census


In [489]:
cr_data_to_union = cr_data_to_union.rename(columns={'ct_digits_cr':'ct','county_cr':'county'})

In [490]:
cr_data_to_union.head()

,ct,county,from
2,201,Anderson,census
3,20201,Anderson,census
4,20202,Anderson,census
5,203,Anderson,census
6,204,Anderson,census


In [502]:
usda_data_to_union['ct']=usda_data_to_union['ct'].astype(str)


In [503]:
usda_data_to_union.dtypes

ct        object
county    object
from      object
dtype: object

In [501]:
cr_data_to_union.dtypes

0
ct        object
county    object
from      object
dtype: object

In [ ]:
joined_usda_cr_tracts = pd.merge(usda_data_to_union,cr_data_to_union, how='full',on=['county'])

In [505]:
joined_usda_cr_tracts.head()

,ct,county_x,from_x,county_y,from_y
0,47001020100,Anderson,usda,NaN,NaN
1,47001020201,Anderson,usda,NaN,NaN
2,47001020202,Anderson,usda,NaN,NaN
3,47001020300,Anderson,usda,NaN,NaN
4,47001020400,Anderson,usda,NaN,NaN


In [497]:
# https://saturncloud.io/blog/how-to-perform-a-union-of-two-pandas-dataframes/
# joined_usda_cr_tracts = pd.concat([usda_data_to_union,cr_data_to_union])